# Projeto Módulo 1 
---

# Resumo

---

# Pontos Principais

---

## Pre processamento
---


## Importações de Bibliotecas

In [1]:
from matplotlib import pyplot as plt
import pandas as pd 
import seaborn as sns 

sns.set_style('darkgrid')
sns.set_palette('hot')

---

## O TabNet

Os dados utilizados neste projeto foram obtidos através da ferramenta TabNet do DataSUS, que esta disponível em: 

* https://datasus.saude.gov.br/informacoes-de-saude-tabnet/

<img src='https://raw.githubusercontent.com/ConradBitt/BootCamp_DataScience/master/Analise%20Dados%20Sa%C3%BAde/projeto%20modulo%201/images/TabNet.png?token=AFR43EQ7J4RD6G4PZUHPS5C7W4X3E' width=90%>

Esta é a interface da ferramenta. Nela estão disponíveis vários dados, inclusive dados que não pertencem diretamente ao DataSUS, como dados Demograficos do IBGE. Os dados que vou utilizar são:

1. AIH aprovadas por Ano/Mês processamento segundo Unidade da Federação.

2. Taxa mortalidade por Ano/Mês processamento segundo Unidade da Federação

3. Valor total por Ano/Mês processamento segundo Unidade da Federação

4. Domicílios por Abastecimento de água, Coleta de lixo e Instalações Sanitárias segundo Unidade da Federação.

5. Media Moradores por domicilios segundo Unidade da Federação

6. Populacao residente segundo Unidade da Federação
    
    **Nota sobre itens 1, 2 e 3:** A partir do processamento de junho de 2012, houve mudança na classificação da natureza e esfera dos estabelecimentos. Com isso, temos que:
      
    1. Até maio de 2012 estas informações estão disponíveis como "Natureza" e "Esfera Administrativa".

    2. De junho de 2012 a outubro de 2015, estão disponíveis tanto como "Natureza" e "Esfera Administrativa", como "Natureza Jurídica" e "Esfera Jurídica".
      
    3. A partir de novembro de 2015, estão disponíveis como "Natureza Jurídica" e "Esfera Jurídica".
    

### Fontes 

* Ministério da Saúde - Sistema de Informações Hospitalares do SUS (SIH/SUS)

* IBGE - Censos Demográfico 2010

Estes dados podem ser obtidos nas seguintes caregorias sublinhadas em vermelho no TabNet: 

<img src='https://raw.githubusercontent.com/ConradBitt/BootCamp_DataScience/master/Analise%20Dados%20Sa%C3%BAde/projeto%20modulo%201/images/Dados_disponiveis_tabnet.png?token=AFR43EX6J5E6HOLDKTRIOTC7W4YAW' width=90%>

## Importações dos Dados

Vou importar os dados que estão dentro da pasta `./dados_datasus/`:

In [2]:
AIH_aprovadas = pd.read_csv('./dados_datasus/AIH aprovadas por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=4,skipfooter=12)
taxa_mortalidade = pd.read_csv('./dados_datasus/Taxa mortalidade por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', decimal=',', skiprows=4,skipfooter=12)
valor_total = pd.read_csv('./dados_datasus/Valor total por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', decimal=',', skiprows=4,skipfooter=12)
internacoes = pd.read_csv('./dados_datasus/Internacoes por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=12)
instalacoes_sanitarias_domicilio = pd.read_csv('./dados_datasus/Domicilios por Instalacoes sanitarias segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
abastecimento_agua_domicilio = pd.read_csv('./dados_datasus/Domicílios por Abastecimento de água segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
coleta_lixo_domicilio = pd.read_csv('./dados_datasus/Domicílios por Coleta de lixo segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
media_moradores_domicilio = pd.read_csv('./dados_datasus/Media MoradoresDomicilios segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python',decimal=',', skiprows=3,skipfooter=8)
populacao_residente = pd.read_csv('./dados_datasus/Populacao residente segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=17)

---
## Tratando os dados

Como o dataset `populacao_residente` do IBGE não tem os números nos índices das macrorregiões, vou adicionar esta series em todos os dataframes no lugar da coluna `Unidade da Federação`, isso vai ser feito criando um dicionário para automatizar a reatribuição das colunas.

Alguns desses dataframes tem séries temporais (`AIH_aprovadas`,`taxa_mortalidade`, `valor_total` e `internacoes`) onde uma pequena parte dos registros estão no tipo `object`, mas são numéricos, então vou transforma-los.

Outros dataframes que armazenam os dados de abastecimento de agua, instalações sanitarias e coleta de lixo são divididas em varios grupos. O tratamento que será feito é dividir cada dataframe em duas categorias: `Rede geral` e `Rede alternativa`.

Vou considerar uma `Rede Geral` os domicílios que tem abastecimento de água por uma rede de canalização, instalação sanitaria por meio de uma rede geral de esgoto ou pluvial e coleta de lixo feita por caçamba ou serviço de limpeza.

Já os domicílios que tem abastecimento de água por meio de poço, nascente, carro-pipa, cisterna, rio, açude, lago ou igarapé. Que usam instalações sanitarias como fossa séptica ou rudimendar, vala, rio, lago ou mar ou outro escoadouro, inclusive quem não tem instalação sanitária. E que descarta o lixo através da queima ou enterrando na propriedade, em terreno baldio ou logradouro, jogado em rio, lago, mar, ou outro destino, como a categoria `Rede Alternativa`.

In [3]:
Unidades_Federacao = populacao_residente['Unidade da Federação']

datasets = {'AIH_aprovadas': AIH_aprovadas,'taxa_mortalidade': taxa_mortalidade,'valor_total': valor_total,'internacoes': internacoes,'abastecimento_agua_domicilio' : abastecimento_agua_domicilio,'instalacoes_sanitarias_domicilio' : instalacoes_sanitarias_domicilio,'coleta_lixo_domicilio' : coleta_lixo_domicilio,'media_moradores_domicilio': media_moradores_domicilio,'populacao_residente': populacao_residente}

for dataset, conteudo in datasets.items():    
    #  Substituindo index
    conteudo['Unidade da Federação'] = Unidades_Federacao
    
    #  Substituindo registros com hífen
    datasets[dataset] = conteudo.replace('-', 'NaN')
    
    #  Referenciando novo conteudo.
    datasets[dataset] = conteudo.set_index('Unidade da Federação')
    
#  Reatribuindo referência as variáveis SIH
AIH_aprovadas = datasets['AIH_aprovadas'].replace('-','NaN').astype(float).fillna(method='ffill').astype(int)
taxa_mortalidade = datasets['taxa_mortalidade'].replace('-','NaN').replace('...','NaN').astype(float).fillna(method='ffill')
valor_total = datasets['valor_total'].replace('-','NaN').astype(float).fillna(method='ffill')
internacoes = datasets['internacoes'].replace('-','NaN').astype(float).fillna(method='ffill').astype(int)

#  Reatribuindo referência as variáveis
instalacoes_sanitarias_domicilio = datasets['instalacoes_sanitarias_domicilio'].replace('-','NaN').astype(float).fillna(0)
abastecimento_agua_domicilio = datasets['abastecimento_agua_domicilio'].replace('-','NaN').astype(float).fillna(0)
coleta_lixo_domicilio = datasets['coleta_lixo_domicilio'].replace('-','NaN').astype(float).fillna(0).astype(int)

abastecimento_agua_domicilio.columns = ['Rede geral','Poço ou nascente','Poço ou nascente fora da propriedade','Carro-pipa', 'Água da chuva armazenada em cisterna','Água da chuva armazenada','Rio, açude, lago ou igarapé','Poço ou nascente na aldeia','Poço ou nascente fora da aldeia','Outra forma','Total']

#  Instalações sanitarias
#instalacoes_sanitarias_domicilio['Rede Geral'] = instalacoes_sanitarias_domicilio[instalacoes_sanitarias_domicilio.columns[0]]
#instalacoes_sanitarias_domicilio['Rede Alternativa'] = instalacoes_sanitarias_domicilio[instalacoes_sanitarias_domicilio.columns[1:7]].sum(axis=1)
#instalacoes_sanitarias_domicilio = instalacoes_sanitarias_domicilio[['Rede Geral', 'Rede Alternativa']]

#  Abastecimento de agua em duas categorias
#abastecimento_agua_domicilio['Rede Geral'] = abastecimento_agua_domicilio[abastecimento_agua_domicilio.columns[0]]
#abastecimento_agua_domicilio['Rede Alternativa'] = abastecimento_agua_domicilio[abastecimento_agua_domicilio.columns[1:10]].sum(axis=1)
#abastecimento_agua_domicilio = abastecimento_agua_domicilio[['Rede Geral', 'Rede Alternativa']]

#  Coleta de Lixo 
#coleta_lixo_domicilio['Rede Geral'] = coleta_lixo_domicilio[coleta_lixo_domicilio.columns[:2]].sum(axis=1)
#coleta_lixo_domicilio['Rede Alternativa'] = coleta_lixo_domicilio[coleta_lixo_domicilio.columns[2:-2]].sum(axis=1)
#coleta_lixo_domicilio = coleta_lixo_domicilio[['Rede Geral','Rede Alternativa']]    

# Dados populacionais do IBGE
media_moradores_domicilio = datasets['media_moradores_domicilio'][['Média_moradores/domicílios']]
populacao_residente = datasets['populacao_residente'][['População_residente']]


datasets = {'AIH_aprovadas': AIH_aprovadas,'taxa_mortalidade': taxa_mortalidade,'valor_total': valor_total,'internacoes': internacoes,'abastecimento_agua_domicilio' : abastecimento_agua_domicilio,'instalacoes_sanitarias_domicilio' : instalacoes_sanitarias_domicilio,'coleta_lixo_domicilio' : coleta_lixo_domicilio,'media_moradores_domicilio': media_moradores_domicilio,'populacao_residente': populacao_residente}

for dataset, conteudo in datasets.items():    
#  Exibindo informações
    print(50*'-')
    print(5*'=', f'> Dataset: {dataset} <', 5*'=')
    print(50*'-')
    display(conteudo.sample(5))
    conteudo.info()
    print('\n')
    
populacao_residente = datasets['populacao_residente']['População_residente']

--------------------------------------------------
===== > Dataset: AIH_aprovadas < =====
--------------------------------------------------


,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Mato Grosso do Sul,8313,9800,10643,11675,11935,13366,12291,11907,12259,11578,...,11886,12483,12562,12968,12097,11787,11235,11361,10737,1816535
Amazonas,5963,9495,10294,10157,9244,10074,10490,11346,10689,10044,...,12277,13552,12714,13742,11514,12684,13209,12454,10058,1687096
Minas Gerais,56796,76614,82076,80630,82213,90776,83298,82979,82677,83909,...,85196,91011,84923,82964,75755,77741,75997,82332,80936,12600919
Pernambuco,17168,24805,30540,34154,33312,31634,30810,32994,32834,30131,...,35420,35961,33322,34553,27590,29072,30628,34095,32414,5125262
Rio Grande do Sul,31564,50686,48092,51743,45037,46363,51970,50623,49070,49870,...,46054,48644,46709,48210,41681,40764,40844,43062,42565,7402737


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Columns: 153 entries, 2008/Jan to Total
dtypes: int64(153)
memory usage: 32.5+ KB


--------------------------------------------------
===== > Dataset: taxa_mortalidade < =====
--------------------------------------------------


,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Rio Grande do Norte,2.32,1.97,2.63,2.68,2.67,3.12,2.63,2.89,2.96,3.09,...,4.40,4.94,4.64,4.50,4.77,5.18,5.99,7.01,7.05,4.14
Distrito Federal,4.81,3.29,2.73,3.20,2.39,2.70,3.00,2.76,3.23,3.29,...,3.08,3.59,3.06,3.28,3.37,3.24,4.53,5.44,6.18,3.34
Amapá,0.69,0.98,0.72,1.62,1.57,1.01,1.56,0.82,1.28,1.53,...,1.82,2.58,1.91,2.63,3.93,6.74,7.06,7.33,4.87,2.10
Mato Grosso do Sul,2.49,3.46,2.86,2.59,3.30,3.09,3.43,3.04,3.36,3.33,...,4.37,4.21,4.45,4.43,5.14,4.59,4.82,5.47,5.86,4.06
Rio de Janeiro,3.41,4.55,4.73,4.31,5.15,4.95,4.94,5.15,5.42,5.34,...,7.79,7.89,7.58,7.83,8.33,10.91,10.30,10.15,9.49,6.90


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Columns: 153 entries, 2008/Jan to Total
dtypes: float64(153)
memory usage: 32.5+ KB


--------------------------------------------------
===== > Dataset: valor_total < =====
--------------------------------------------------


...

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 8 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Coletado por serviço de limpeza             27 non-null     int64
 1   Coletado por caçamba de serviço de limpeza  27 non-null     int64
 2   Queimado (na propriedade)                   27 non-null     int64
 3   Enterrado (na propriedade)                  27 non-null     int64
 4   Jogado em terreno baldio ou logradouro      27 non-null     int64
 5   Jogado em rio, lago ou mar                  27 non-null     int64
 6   Outro destino                               27 non-null     int64
 7   Total                                       27 non-null     int64
dtypes: int64(8)
memory usage: 1.9+ KB


--------------------------------------------------
===== > Dataset: media_moradores_domicilio < =====
-----------------------------

,Média_moradores/domicílios
Unidade da Federação,
Paraíba,3.1
Rio Grande do Sul,2.8
Acre,3.6
Sergipe,3.2
Amazonas,3.9


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Média_moradores/domicílios  27 non-null     float64
dtypes: float64(1)
memory usage: 432.0+ bytes


--------------------------------------------------
===== > Dataset: populacao_residente < =====
--------------------------------------------------


,População_residente
Unidade da Federação,
Ceará,8472231
Paraíba,3794080
Rio Grande do Sul,10860160
Espírito Santo,3447503
Rio de Janeiro,15684755


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   População_residente  27 non-null     int64
dtypes: int64(1)
memory usage: 432.0+ bytes




# Análise dos Dados
---

Grafico tem motivo

Uma tabela pode ser melhor que um gráfico 

Gera a hipótese, faz o código, analise do resultado.

Não testar vários modelos num mesmo modelo. Lucidez na escolha de modelo.

Ser claro com as hipóteses e objetivos.


--- 
## Cenarios sanitarios com respeito a instalções, abastecimento de agua e coleta de lixo

In [135]:
abastecimento_agua_por_populacao_residente.head()

Unidade da Federação,Rondônia,Acre,Amazonas,Roraima,Pará,Amapá,Tocantins,Maranhão,Piauí,Ceará,...,Espírito Santo,Rio de Janeiro,São Paulo,Paraná,Santa Catarina,Rio Grande do Sul,Mato Grosso do Sul,Mato Grosso,Goiás,Distrito Federal
Rede geral,11.549708,13.024452,15.190323,22.349817,12.098671,13.624391,24.042563,17.023499,19.361527,21.559245,...,26.783530,28.271254,29.909335,27.391802,26.662142,28.284252,26.530051,22.687785,25.484900,29.144150
Poço ou nascente,17.033610,10.822228,3.028737,3.428440,9.290685,9.254216,4.813240,4.208930,2.566769,2.610422,...,4.123593,3.798599,1.140741,2.761499,4.304666,3.731612,4.321662,6.357085,5.635417,0.931079
Poço ou nascente fora da propriedade,1.087892,1.261965,2.265308,0.443191,2.002487,1.045792,0.650954,3.548712,2.806654,1.632982,...,0.887077,0.847492,0.261724,0.845003,1.610704,0.953163,0.656951,0.804301,0.620865,0.504457
Carro-pipa,0.011720,0.175739,0.008033,0.008075,0.062762,0.011348,0.009440,0.109000,0.455370,0.373715,...,0.018709,0.184676,0.022206,0.003319,0.002528,0.015046,0.018087,0.065690,0.020971,0.007126
Água da chuva armazenada em cisterna,0.000922,0.008960,0.028629,0.001663,0.008361,0.015024,0.081887,0.010547,0.645066,0.252838,...,0.008702,0.016468,0.001754,0.001622,0.005089,0.010166,0.004680,0.006679,0.103546,0.011402
